[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1M0n67UrVkcu5quSUROx14z74kcCUHpGT?usp=sharing)

In [ ]:
!pip install git+https://github.com/scikit-learn-contrib/hdbscan.git
!pip install InstructorEmbedding bertopic chart_studio

In [ ]:
import re
import pandas as pd

from bertopic import BERTopic
from google.colab import files
from bertopic.backend import BaseEmbedder
from InstructorEmbedding import INSTRUCTOR
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
files.upload()

Saving data_organic_cleaned.parquet to data_organic_cleaned.parquet


In [ ]:
class CustomEmbedder(BaseEmbedder):
    def __init__(self, embedding_model: str='hkunlp/instructor-large', instruction: str="Represent this text:"):
        super().__init__()
        self.embedding_model = INSTRUCTOR(embedding_model)
        self.instruction = instruction

    def embed(self, documents, verbose=False):
        embeddings = self.embedding_model.encode([[self.instruction, doc] for doc in documents])
        return embeddings



class TopicModel:
  def __init__(self, instruction: str, min_topic_size: int):

    self.topics = None
    self.probs = None

    self.model = BERTopic(
                  language= "english",
                  top_n_words = 10,
                  n_gram_range = (1, 1),
                  min_topic_size = min_topic_size,
                  low_memory = False,
                  seed_topic_list = None,
                  embedding_model=CustomEmbedder(),
                  vectorizer_model=CountVectorizer(ngram_range=(1, 1), stop_words="english"),
                  umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42),
                  representation_model = None,
                  verbose=False,
                  )


  def run_per_class(self, docs, classes):

    if not self.topics == None or self.probs == None:
      self.topics, self.probs = self.model.fit_transform(docs)

    return self.model.topics_per_class(docs, classes=classes)


  def run_dynamic(self, docs, timestamps):

    if not self.topics == None or self.probs == None:
      self.topics, self.probs = self.model.fit_transform(docs)

    return self.model.topics_over_time(docs, timestamps, nr_bins=20)


## Paid

In [ ]:
df = pd.read_parquet("data_cleaned.parquet").query("partisanship != 'unknown'").reset_index(drop=True)
df['ad_creative_body'] = df['ad_creative_body'].map(lambda x: re.sub(r'http\S+', '', x))
df = df.drop_duplicates(subset="ad_creative_body")

In [ ]:
temp_df = df.drop_duplicates(subset="ad_creative_body")
instruction = "Represent this Facebook political ad on trangender issues:"

tm = TopicModel(instruction=instruction, min_topic_size=8)

topics_per_class = tm.run_per_class(docs=temp_df["ad_creative_body"], classes=temp_df["partisanship"])
topics_over_time = tm.run_dynamic(docs=temp_df["ad_creative_body"], timestamps=temp_df["ad_delivery_start_time"])

In [ ]:
fig = tm.model.visualize_topics_per_class(topics_per_class, top_n_topics=11)
fig.show()

In [ ]:
tm.model.get_topic_info()[1:11]

Topic  Count                                                 Name  \
1      0    120                       0_trans_youth_transgender_care   
2      1     36  1_turningpointaction_democrats_leftist_conservative   
3      2     34                        2_pride_trans_lgbtq_community   
4      3     19                           3_sick_ideology_money_read   
5      4     19                                  4_bud_30_genders_pm   
6      5     15                           5_montana_zephyr_rep_zooey   
7      6     12           6_transgenderism_catholic_catholics_social   
8      7     10                 7_abc_teens_conversation_transgender   

                                                                                              Representation  \
1                              [trans, youth, transgender, care, need, state, help, people, support, sports]   
2  [turningpointaction, democrats, leftist, conservative, republican, woke, pride, lgbtq, transgender, drag]   
3                   [pride, trans, lgbtq, community, rights, queer, stonewall, transgender, people, lgbtqia]   
4                             [sick, ideology, money, read, years, john, started, studies, gender, children]   
5                                             [bud, 30, genders, pm, men, women, father, razors, light, com]   
6              [montana, zephyr, rep, zooey, lawmaker, republicans, gop, transgender, speaking, legislative]   
7             [transgenderism, catholic, catholics, social, teaching, respond, read, causes, view, articles]   
8                        [abc, teens, conversation, transgender, world, hulu, special, news, stories, watch]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
fig = tm.model.visualize_topics_over_time(topics_over_time, top_n_topics=11)
fig.show()

In [ ]:
topic_df = tm.model.get_document_info(temp_df["ad_creative_body"])
topic_df = df[["ad_creative_body", 'spend', 'impressions']].merge(topic_df[["Document", "Name"]], left_on="ad_creative_body", right_on="Document", how="left")
topic_df = topic_df.groupby("Name").sum().sort_values(by="impressions", ascending=False).reset_index()
topic_df["impressions_per_dollar"] = topic_df["impressions"] / topic_df["spend"]

In [ ]:
topic_df

,Name,spend,impressions,impressions_per_dollar
0,0_trans_youth_transgender_care,11540.0,830880,72.000000
1,3_sick_ideology_money_read,5040.5,646981,128.356512
2,4_bud_30_genders_pm,4340.5,598481,137.882963
3,2_pride_trans_lgbtq_community,5083.0,405966,79.867401
4,-1_desantis_books_banned_students,4022.5,256445,63.752641
5,1_turningpointaction_democrats_leftist_conservative,2282.0,235464,103.183173
6,7_abc_teens_conversation_transgender,495.0,41990,84.828283
7,5_montana_zephyr_rep_zooey,942.5,32485,34.466844
8,6_transgenderism_catholic_catholics_social,594.0,7988,13.447811


## Organic

In [ ]:
df_organic = pd.read_parquet("data_organic_cleaned.parquet").query("partisanship_simplified != 'unknown'").reset_index(drop=True)
df_organic['message'] = df_organic['message'].map(lambda x: re.sub(r'http\S+', '', x))
df_organic = df_organic.drop_duplicates(subset="message")

In [ ]:
df_organic["partisanship_simplified"].value_counts()

center    7572
right     5420
left      4893
Name: partisanship_simplified, dtype: int64

In [ ]:
instruction = "Represent this Facebook post on trangender issues:"

tm = TopicModel(instruction=instruction, min_topic_size=150)



load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
df_organic["partisanship"].value_counts()

Center            7633
Far Right         4167
Slightly Left     2559
Far Left          2393
Slightly Right    1328
Name: partisanship, dtype: int64

In [ ]:
topics_over_time = tm.run_dynamic(docs=df_organic["message"], timestamps=df_organic["day"])

In [ ]:
topics_per_class = tm.run_per_class(docs=df_organic["message"], classes=df_organic["partisanship_simplified"])

In [ ]:
#tm.model.save("organic_topic_model", serialization="safetensors", save_ctfidf=True,)

In [ ]:
tm.model.get_topic_info().head(40)

Topic  Count                                         Name  \
0      -1   8790                    -1_trans_people_said_like   
1       0    453                  0_trump_biden_ukraine_china   
2       1    302                  1_pride_month_june_bisexual   
3       2    274                 2_zephyr_zooey_montana_floor   
4       3    236            3_target_merchandise_stores_items   
5       4    208                   4_bud_light_anheuser_busch   
6       5    205                    5_texas_abbott_14_hormone   
7       6    176         6_bills_introduced_legislatures_anti   
8       7    176                7_girls_house_sports_athletic   
9       8    173                      8_senate_veto_vote_desk   
10      9    167              9_dylan_mulvaney_influencer_bud   
11     10    161           10_nashville_shooting_mass_shooter   
12     11    156    11_amplifier_audioxpress_mosquito_ability   
13     12    153                12_topless_white_montoya_rose   
14     13    153                     13_love_good_thank_brave   
15     14    151         14_visibility_gathered_rally_marched   
16     15    150                      15_rock_kid_cases_rifle   
17     16    147           16_prison_prisons_inmates_sexually   
18     17    141  17_transgenderism_lobby_neill_transhumanism   
19     18    136      18_dodgers_sisters_indulgence_perpetual   
20     19    134                      19_sales_light_bud_beer   
21     20    131          20_ix_administration_title_proposed   
22     21    125         21_graduation_dress_mississippi_wear   
23     22    121            22_proposal_forbidden_ix_enacting   
24     23    116            23_gaines_riley_swimmer_francisco   
25     24    115         24_advantage_athletes_sports_compete   
26     25    113             25_teacher_names_religious_fired   
27     26    111          26_board_bathrooms_students_meeting   
28     27    110            27_rowling_potter_harry_radcliffe   
29     28    107               28_shirt_genders_morrison_liam   
30     29    107             29_benestad_medicine_health_care   
31     30    106               30_nebraska_abortion_pillen_12   
32     31    102           31_mcbride_delaware_sarah_congress   
33     32    101                   32_god_church_bible_christ   
34     33     99                 33_monica_lysette_film_trace   
35     34     99          34_manifesto_hale_shooter_nashville   
36     35     98         35_montana_lawmaker_silenced_censure   
37     36     98           36_absence_marketing_leave_oversaw   
38     37     95               37_gaines_riley_swimmer_thomas   
39     38     91                  38_ceo_anheuser_busch_dylan   

                                                                                            Representation  \
0                         [trans, people, said, like, transgender, gender, just, don, children, community]   
1                    [trump, biden, ukraine, china, president, hunter, faces, russia, criticizes, russian]   
2                          [pride, month, june, bisexual, city, lesbian, gay, celebrate, community, queer]   
3                         [zephyr, zooey, montana, floor, rep, lawmaker, barred, session, house, silenced]   
4            [target, merchandise, stores, items, clothing, tuck, swimsuits, products, retailer, designer]   
5                                  [bud, light, anheuser, busch, drink, beer, fratty, trying, brew, stock]   
6                        [texas, abbott, 14, hormone, greg, puberty, blockers, therapies, minors, largest]   
7              [bills, introduced, legislatures, anti, targeting, laws, legislation, lgbtq, tallies, data]   
8                   [girls, house, sports, athletic, women, teams, competing, passed, athletes, federally]   
9                                   [senate, veto, vote, desk, unlikely, advance, house, 203, 219, passed]   
10                    [dylan, mulvaney, influencer, bud, light, lite, beer, miller, partnership, backlash]   
11             [na

In [ ]:
fig = tm.model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
fig.show()

In [ ]:
fig = tm.model.visualize_topics_over_time(topics_over_time, top_n_topics=10)
fig.show()

In [ ]:
topic_df = tm.model.get_document_info(df_organic["message"])
topic_df = df_organic[["message", 'engagement']].merge(topic_df[["Document", "Name"]], left_on="message", right_on="Document", how="left")
topic_df = topic_df.groupby("Name").sum().sort_values(by="engagement", ascending=False).reset_index()

In [ ]:
topic_df.head(10)

,Name,engagement
0,-1_trans_people_said_like,1028756
1,15_rock_kid_cases_rifle,91100
2,37_gaines_riley_swimmer_thomas,89119
3,94_lingerie_adidas_klein_calvin,83529
4,50_tritt_travis_hospitality_music,71890
5,69_billion_value_anheuser_busch,60390
6,13_love_good_thank_brave,47329
7,4_bud_light_anheuser_busch,46530
8,384_patrons_toss_backtracked_hatred,44771
9,2_zephyr_zooey_montana_floor,32470
